In [151]:
import requests
import pysondb
import json
import pandas as pd
import random

In [30]:
db = pysondb.db.getDb('citydb.json')

In [38]:
file = open("mirror.txt")
token = file.read()

In [67]:
f2 = pd.read_csv("city_data.csv")
f2.head()

,city,cityLabel,population,wiki url
0,Q11725,Chongqing,32054159,https://en.wikipedia.org/wiki/Chongqing
1,Q1353,Delhi,26495000,https://en.wikipedia.org/wiki/Delhi
2,Q8686,Shanghai,24870895,https://en.wikipedia.org/wiki/Shanghai
3,Q956,Beijing,21893095,https://en.wikipedia.org/wiki/Beijing
4,Q30002,Chengdu,20937757,https://en.wikipedia.org/wiki/Chengdu


In [133]:
f3 = f2[f2.isna().any(axis=1)]
f3

,city,cityLabel,population,wiki url
2162,Q11544714,Sidoarjo,223231,NaN
2530,Q21414421,Oviedo,185902,NaN
2556,Q16898113,Oradea,183105,NaN
2567,Q3344946,Braga,181494,NaN
2651,Q62866867,Deo,173216,NaN
2677,Q12185424,Jubail,170770,NaN
2729,Q12185424,Jubail,167008,NaN
2930,Q3133884,Palo Negro,153706,NaN
3064,Q119806208,City of Francistown,147122,NaN
3205,Q3018561,Apeldoorn,138945,NaN


In [135]:
f2 = f2[~f2.isna().any(axis=1)]
f2

,city,cityLabel,population,wiki url
0,Q11725,Chongqing,32054159,https://en.wikipedia.org/wiki/Chongqing
1,Q1353,Delhi,26495000,https://en.wikipedia.org/wiki/Delhi
2,Q8686,Shanghai,24870895,https://en.wikipedia.org/wiki/Shanghai
3,Q956,Beijing,21893095,https://en.wikipedia.org/wiki/Beijing
4,Q30002,Chengdu,20937757,https://en.wikipedia.org/wiki/Chengdu
...,...,...,...,...
4217,Q889344,Boconó,100240,https://en.wikipedia.org/wiki/Boconó
4218,Q622633,Nampa,100200,"https://en.wikipedia.org/wiki/Nampa,_Idaho"
4219,Q1714,Lamitan,100150,https://en.wikipedia.org/wiki/Lamitan
4220,Q838785,Temecula,100097,"https://en.wikipedia.org/wiki/Temecula,_Califo..."


In [16]:
my_headers = headers={'accept': 'application/json', 'Authorization': 'Bearer ' + token, 'content-type': 'application/json'}

In [61]:
r = requests.get(url_test, headers=my_headers)

In [66]:
with pd.option_context('display.max_rows', None,
                       'display.max_columns', None,
                       'display.precision', 3,
                       ):
    print(f2)

            city                            cityLabel  population  \
0         Q11725                            Chongqing    32054159   
1          Q1353                                Delhi    26495000   
2          Q8686                             Shanghai    24870895   
3           Q956                              Beijing    21893095   
4         Q30002                              Chengdu    20937757   
5         Q16572                            Guangzhou    18676605   
6         Q15174                             Shenzhen    17494398   
7          Q1354                                Dhaka    16800000   
8           Q406                             Istanbul    15462452   
9          Q1156                               Mumbai    15414288   
10         Q8673                                Lagos    15070000   
11         Q8660                              Karachi    14910352   
12         Q1490                                Tokyo    14264798   
13        Q11736                  

In [161]:
def fetch_wiki_article(url, cityLabel):
    new_url = 'https://api.enterprise.wikimedia.com/v2/structured-contents/' + url.split("https://en.wikipedia.org/wiki/")[1]
    r = requests.get(new_url, headers=my_headers)
    data = r.json()
    cityfile = cityLabel + str(random.randint(1,1000))+ '.json'
    with open('CITYJSON/' + cityfile, 'w') as f:
        json.dump(data, f)
    return None
    

In [159]:
#f2.apply(lambda x: fetch_wiki_article(x['wiki url'], x['cityLabel']), axis=1)
'CITYJSON/' + 'Alexandria.json' + str(random.randint(1,1000))

'CITYJSON/Alexandria.json477'

In [140]:
df = pd.concat(g for _, g in f2.groupby("cityLabel") if len(g) > 1)
with pd.option_context('display.max_rows', None,
                       'display.max_columns', None,
                       'display.precision', 3,
                       ):
    print(df)

           city             cityLabel  population  \
1154    Q337539  10th of Ramadan City      500000   
1973    Q337539  10th of Ramadan City      250000   
473       Q3787                 Abuja     1568853   
580       Q3787                 Abuja     1235880   
715       Q3787                 Abuja      979876   
832       Q3787                 Abuja      776298   
3979      Q3787                 Abuja      107169   
2484   Q1141858                Ajloun      190200   
3023   Q1141858                Ajloun      148870   
145         Q87            Alexandria     4870000   
2841        Q88            Alexandria      159467   
116     Q324293                Anyang     5477614   
1039     Q42120                Anyang      573296   
1063     Q42120                Anyang      548372   
2728    Q626199                  Arar      167057   
3193    Q626199                  Arar      139608   
1362     Q17943             Arlington      394266   
2047  Q49279759             Arlington      238

In [154]:
msk = df.groupby(['cityLabel', 'city'])['city'].transform('size') > 1
df1 = df[msk]
with pd.option_context('display.max_rows', None,
                       'display.max_columns', None,
                       'display.precision', 3,
                       ):
    print(df1)

           city             cityLabel  population  \
1154    Q337539  10th of Ramadan City      500000   
1973    Q337539  10th of Ramadan City      250000   
473       Q3787                 Abuja     1568853   
580       Q3787                 Abuja     1235880   
715       Q3787                 Abuja      979876   
832       Q3787                 Abuja      776298   
3979      Q3787                 Abuja      107169   
2484   Q1141858                Ajloun      190200   
3023   Q1141858                Ajloun      148870   
1039     Q42120                Anyang      573296   
1063     Q42120                Anyang      548372   
2728    Q626199                  Arar      167057   
3193    Q626199                  Arar      139608   
2047  Q49279759             Arlington      238643   
2286  Q49279759             Arlington      207627   
3493    Q590849                Arvada      124402   
3994    Q590849                Arvada      106433   
1723    Q486913                Atyrau      295

In [158]:
df3 = df[~df.isin(df1)].dropna()
df3

,city,cityLabel,population,wiki url
145,Q87,Alexandria,4870000.0,https://en.wikipedia.org/wiki/Alexandria
2841,Q88,Alexandria,159467.0,"https://en.wikipedia.org/wiki/Alexandria,_Virg..."
116,Q324293,Anyang,5477614.0,https://en.wikipedia.org/wiki/Anyang
1362,Q17943,Arlington,394266.0,"https://en.wikipedia.org/wiki/Arlington,_Texas"
957,Q1524,Athens,643452.0,https://en.wikipedia.org/wiki/Athens
...,...,...,...,...
4143,Q1646181,Worcester,101891.0,"https://en.wikipedia.org/wiki/Worcester,_England"
138,Q363326,Yichun,5007702.0,"https://en.wikipedia.org/wiki/Yichun,_Jiangxi"
624,Q349976,Yichun,1148126.0,"https://en.wikipedia.org/wiki/Yichun,_Heilongj..."
105,Q571918,Yulin,5796766.0,"https://en.wikipedia.org/wiki/Yulin,_Guangxi"


In [162]:
df3.apply(lambda x: fetch_wiki_article(x['wiki url'], x['cityLabel']), axis=1)

145     None
2841    None
116     None
1362    None
957     None
        ... 
4143    None
138     None
624     None
105     None
209     None
Length: 144, dtype: object